## 7.3 - Solar flare [dataset](https://archive.ics.uci.edu/ml/datasets/Solar+Flare)

En este conjunto de datos, tendremos dos ficheros relativos a `data`, cuya primera fila serán las especificaciones temporales, por lo que deberemos quitarla, y además en los registros de datos las variables no vienen delimitadas por `','`, si no por espacios en blanco:

### La manera de Andrés (no me funciona)

In [ ]:
# Movemos el directorio activo a una nueva localización para este dataset
## Retrocedemos dos niveles
%cd ..
%cd ..
## Creamos carpeta
!mkdir /content/solar_flare_dataset
## Movemos directorio activo
%cd /content/solar_flare_dataset
# Descargamos los ficheros que contienen los datos a nuestro directorio activo
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/solar-flare/flare.data1
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/solar-flare/flare.data2
# Descargamos la metadata asociada al conjunto de datos
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/solar-flare/flare.names
# Leemos datos
## Leemos primer fichero de datos
with open(os.path.join(os.getcwd(),'flare.data1'),'r') as f:
    data1 = f.read().splitlines() # Dividimos el texto por saltos de línea
    data1 = [elem.split(' ') for elem in data1 if elem!=''] # Dividimos cada línea por las comas y removemos líneas vacías
    data1 = data1[1:] # Quitamos la línea de metadata temporal
## Leemos segundo fichero de datos
with open(os.path.join(os.getcwd(),'flare.data2'),'r') as f:
    data2 = f.read().splitlines() # Dividimos el texto por saltos de línea
    data2 = [elem.split(' ') for elem in data2 if elem!=''] # Dividimos cada línea por las comas y removemos líneas vacías
    data2 = data2[1:] # Quitamos la línea de metadata temporal
## Combinamos ambas listas
data = data1+data2
# Leemos metadata
with open(os.path.join(os.getcwd(),'flare.names'),'r') as f:
    metadata = f.read().splitlines()
## Regex
regex_fn = lambda text: re.findall('^\s+[0-9]+\.{1}\s{1}[a-zA-Z- ]+', text)
reg_text_fn = lambda text : re.findall('[a-zA-Z-]+', text)
metadata_list = [regex_fn(elem)[0].strip() for elem in metadata if regex_fn(elem)]
col_names = [reg_text_fn(elem)[0] for elem in metadata_list if reg_text_fn(elem)]
# Construimos el objeto pd.DataFrame
df = pd.DataFrame(data=data, columns=col_names)

### La manera de Demetrio

https://archive.ics.uci.edu/ml/machine-learning-databases/solar-flare/flare.data1 <br>
https://archive.ics.uci.edu/ml/machine-learning-databases/solar-flare/flare.data2

In [ ]:
url1 = 'https://archive.ics.uci.edu/ml/machine-learning-databases/solar-flare/flare.data1'
url2 = 'https://archive.ics.uci.edu/ml/machine-learning-databases/solar-flare/flare.data2'
url3 = 'https://archive.ics.uci.edu/ml/machine-learning-databases/solar-flare/flare.names'

In [ ]:
# Vemos qué tenemos dentro de cada variable
# data1
import requests
response = requests.get(url1)
data1 = response.text
data1

Se ve que los datos están separados por espacios en vez de por comas

In [ ]:
# data2
response = requests.get(url2)
data2 = response.text
data2

In [ ]:
#creamos un dataframe con los datos de data1 y data2 
import pandas as pd
import io

#Leemos los datos como un csv de string separado por espacios
data1df = pd.read_csv(io.StringIO(data1), sep=' ')
data2df = pd.read_csv(io.StringIO(data2), sep=' ')

#vemos que tenemos en data1
data1df.head()

In [ ]:
#vemos que tenemos en data2
data2df.head()

In [ ]:
#podemos ver el mismo problema en el nombre de las columnas, ya que usaron un texto como nombre de columna
#vamos a quitar ese texto de los archivos
#recuerden que el texto lo podemos ver arriba cuando hicimos el print es *******DATA1;	1969	FLARE	DATA	Unnamed: 4	(02/13/69	to	03/27/69)	*******
#y 	*******	DATA2;	1978	FLARE	DATA	(08/19/78	to	12/23/78)	*****

#vamos a quitar el texto de las columnas creando nuevos dataframes a partir del texto que queremos quitar
data1 
#al hacer el print vemos como despues del texto que queremos quitar empiezan los datos

In [ ]:
#mediante expresiones regulares vamos a quitar el texto que no queremos
import re

#esto significa que vamos a quitar todo lo que este antes de DATA1; y todo lo que este despues de DATA1; y antes de un salto de linea, y lo vamos a 
#sustituir por "" en data1 (y lo mismo para data2)
data1 = re.sub(r'.*DATA1;.*\n', '', data1) 
data2 = re.sub(r'.*DATA2;.*\n', '', data2) 

In [ ]:
data1

In [ ]:
data2

In [ ]:
#podemos crear un df ahora con los datos limpios, separados por espacios

#Volvemos a hacer lo mismo de antes, pero ahora con los datos 1 y 2 transformados con la operación anterior.
data1df = pd.read_csv(io.StringIO(data1), sep=' ', header=None)   #header=none para que no tome la primera fila como nombre de columna
data2df = pd.read_csv(io.StringIO(data2), sep=' ' , header=None)  #.io para que lo tome como un archivo de texto

In [ ]:
data1df.head(2)

In [ ]:
data2df.head(2)

Ahora que hemos limpiado los datos, vamos a trabajar con el archivo de nombres

In [ ]:
response = requests.get(url3)
data3 = response.text
print(data3) #vemos que las columnas estan en el punto 7

¿Cómo accedemos a los nombres de las columnas, que están en el apartado 7 de data3?

In [ ]:
#extraemos la info del punto 7. "Attribute Information" , vemos que tenemos el 7 con un punto, eso es bueno para localizar
#Vamos a quedarnos con lo que exista despues del punto 7 y antes del punto 8
data3 = re.search(r'7\..*8\.', data3, re.DOTALL).group(0) #re.DOTALL para que tome en cuenta los saltos de linea

In [ ]:
print(data3) #Vemos la información que ha extraído de data 3

Ahora queremos acceder a cada uno de los puntos para convertirlos en los encabezados del df.

In [ ]:
#creamos un patron para extraer los nombres de las columnas

#\d+ para que tome cualquier numero de digitos
#\s+ para que tome cualquier espacio en blanco
#[^\(\n]+ para que tome cualquier caracter que no sea un parentesis o un salto de linea

pattern = re.compile(r'\d+\.\s+([^\(\n]+)') 

# Encontramos todas las coincidencias
columns = pattern.findall(data3) #findall 

# Mostramos los nombres de las columnas con un bucle
for col in columns:
    print(col.strip())

In [ ]:
columns

In [ ]:
#ahora eliminamos la palabra 'Number' de las últimas 3 filas y Attribute Information: de la primera fila de nuestra nueva variable columns
columns = [re.sub(r'Number', '', col) for col in columns] #re.sub para reemplazar 'Number' por nada en "col"
columns = [re.sub(r'Attribute Information:', '', col) for col in columns] #re.sub para reemplazar 'Attribute Information:' por nada en "col"

In [ ]:
columns #vemos que ya no tenemos 'Number' ni 'Attribute Information:', ahora podemos limpiar todo de caracteres especiales

In [ ]:
#vamos a limpiar los nombres de las columnas eliminando los espacios en blanco y caracteres especiales
columns = [re.sub(r'[^\w]', '', col) for col in columns] #re.sub para reemplazar cualquier caracter que no sea una letra o un numero por nada en (cada) col
columns = [col.lower() for col in columns] #pasamos todo a minusculas

columns #vemos que ya tenemos los nombres de las columnas limpios

In [ ]:
#eliminamos el primer elemento de la lista ya que esta vacio
if columns and columns[0] == '':
    columns.pop(0)

In [ ]:
columns

Con esto ya hemos obtenido los nombres de todas las columnas de data1 y data2, por lo que vamos a unirlos y a indicar sus columnas

In [ ]:
df = pd.concat([data1df, data2df], axis=0) #unimos ambas bases de datos. axis=0 para concatenar por filas
df.columns = columns #asignamos los nombres de las columnas
df.head() #vemos que ya tenemos nuestro df limpio

### Análisis estadístico de los datos (Explayarse todo lo que uno quiera)